In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
from gensim import corpora, models
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Ensure required NLTK resources are available
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import sqlite3
import pandas as pd

def connect_to_db(db_path):
    """Establish a connection to the SQLite database."""
    conn = sqlite3.connect(db_path)
    return conn

def load_table_to_dataframe(conn, table_name):
    """Load a table from the SQLite database into a Pandas DataFrame."""
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, conn)
    return df

def create_text_column(df):
    """Create a new column 'TEXT' by concatenating specific columns."""
    df['TEXT'] = df['title'] + ' ' + df['meta_description'] + ' ' + df['description'] + ' ' + df['body']
    df['TEXT'] = df['TEXT'].fillna('')
    return df

# DB path
db_path = r"../data/articles.sqlite"

# Connect to the database
conn = connect_to_db(db_path)

# Load the table into a DataFrame
table_name = "article"
df = load_table_to_dataframe(conn, table_name)

# Create the 'TEXT' column
df = create_text_column(df)

# Close the database connection
conn.close()

# Display the resulting DataFrame
print(df.columns)
print(df.head())

Index(['id', 'url', 'title', 'meta_description', 'description', 'date', 'tags',
       'author', 'body', 'status', 'TEXT'],
      dtype='object')
   id                                                url  \
0   1  https://www.elespectador.com/salud/no-es-una-s...   
1   2  https://www.elespectador.com/salud/corte-orden...   
2   3  https://www.elespectador.com/politica/la-adver...   
3   4  https://www.elespectador.com/salud/supersalud-...   
4   5  https://www.elespectador.com/salud/reforma-a-l...   

                                               title  \
0  No es una, son tres las reformas a la salud qu...   
1  Corte ordena al Minsalud pagar saldos pendient...   
2  “El que no haga caso, se va”: la advertencia d...   
3  Supersalud interviene a la EPS Sanitas, con má...   
4  Reforma a la salud: aprobaron el 49%, pero fal...   

                                    meta_description  \
0  A la propuesta del Gobierno se suma una de la ...   
1  El Ministerio de Salud tendrá un plazo de

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andres\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andres\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Andres\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Documentación del Código

Este script implementa una aplicación web interactiva utilizando Dash para explorar artículos de noticias agrupados por temas utilizando un modelo LDA (Latent Dirichlet Allocation). La aplicación permite a los usuarios seleccionar un tema y visualizar los artículos relacionados con dicho tema, ordenados por fecha.

## Librerías Utilizadas

- `dash`: Framework para crear aplicaciones web interactivas en Python.
- `plotly.express`: Utilizado para crear gráficos interactivos.
- `pandas`: Utilizado para la manipulación de datos en DataFrames.
- `gensim`: Utilizado para el procesamiento de texto y modelado de tópicos LDA.
- `nltk`: Utilizado para la tokenización, eliminación de stopwords y lematización.
- `dateutil`: Utilizado para parsear fechas en diferentes formatos.
- `os`: Utilizado para interactuar con el sistema de archivos.

## Preprocesamiento del Texto

### `preprocess_text(text)`
Esta función realiza el preprocesamiento del texto, que incluye:

- **Tokenización:** Divide el texto en palabras (tokens).
- **Filtrado:** Elimina tokens que no sean alfabéticos y stopwords en español.
- **Lematización:** Reduce cada palabra a su forma base.

- **Parámetros:**
  - `text` (str): Texto a preprocesar.
  
- **Retorna:**
  - `tokens` (list): Lista de tokens preprocesados.

## Parseo de Fechas en Español

### `parse_spanish_date(date_str)`
Esta función convierte una cadena de texto con fecha en español a un objeto de fecha en formato `datetime`.

- **Parámetros:**
  - `date_str` (str): Cadena de texto que contiene la fecha en español.
  
- **Retorna:**
  - `parsed_date` (datetime): Objeto `datetime` que representa la fecha parseada.

## Creación del Modelo LDA

### `identify_topics_by_frequent_bigrams(lda_model, corpus, preprocessed_texts, num_bigrams=2)`
Identifica los temas generados por el modelo LDA mediante los bigramas más frecuentes en los documentos asociados a cada tema.

- **Parámetros:**
  - `lda_model` (gensim.models.LdaModel): Modelo LDA entrenado.
  - `corpus` (list): Corpus en formato BoW (Bag of Words) de los documentos.
  - `preprocessed_texts` (pandas.Series): Serie de textos preprocesados.
  - `num_bigrams` (int): Número de bigramas más frecuentes a identificar por tema.
  
- **Retorna:**
  - `topic_labels` (dict): Diccionario que mapea el ID del tema a su label basado en bigramas.
  - `grouped_documents` (dict): Diccionario que agrupa los IDs de documentos por tema.

### Creación y Guardado del Modelo LDA

El script verifica si un modelo LDA entrenado ya existe en el disco (`lda_model_path`). Si es así, lo carga; de lo contrario, entrena un nuevo modelo LDA utilizando el corpus y lo guarda.

## Configuración de la Aplicación Dash

### `app.layout`
Define el diseño de la aplicación Dash, que incluye:

- **Título de la aplicación.**
- **Dropdown:** Permite seleccionar un tema basado en los bigramas más frecuentes.
- **Div:** Contenedor para mostrar los artículos filtrados.

### `update_news_display(selected_topic)`
Esta función es un callback que se ejecuta cada vez que se selecciona un tema en el dropdown. Filtra los artículos por el tema seleccionado y muestra los resultados ordenados por fecha.

- **Parámetros:**
  - `selected_topic` (str): El tema seleccionado en el dropdown.
  
- **Retorna:**
  - `news_articles` (list): Lista de componentes HTML que muestran los artículos de noticias filtrados.

## Ejecución de la Aplicación

La aplicación se ejecuta utilizando el servidor de Dash:

```python
if __name__ == '__main__':
    app.run_server(debug=True)


In [3]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
from gensim import corpora, models
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer
from dateutil import parser
import os

# Ensure required NLTK resources are available
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Initialize stop words and lemmatizer
stop_words = set(stopwords.words('spanish'))
lemmatizer = WordNetLemmatizer()

# Preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# Custom function to parse the Spanish date strings
def parse_spanish_date(date_str):
    # Handle None or NaN values
    if pd.isnull(date_str):
        return pd.NaT
    
    # Replace month names in Spanish with numbers
    months = {
        "enero": "01",
        "febrero": "02",
        "marzo": "03",
        "abril": "04",
        "mayo": "05",
        "junio": "06",
        "julio": "07",
        "agosto": "08",
        "septiembre": "09",
        "octubre": "10",
        "noviembre": "11",
        "diciembre": "12"
    }
    
    date_str = date_str.lower()
    
    for month, num in months.items():
        date_str = date_str.replace(month, num)
    
    # Now try to parse the date using dateutil.parser
    try:
        parsed_date = parser.parse(date_str, dayfirst=True)
        # Remove timezone info if present
        if parsed_date.tzinfo is not None:
            parsed_date = parsed_date.replace(tzinfo=None)
    except ValueError:
        parsed_date = pd.NaT  # Not a Time if parsing fails
    
    return parsed_date

# Apply the custom parsing function to the 'date' column
df['DATE'] = df['date'].apply(parse_spanish_date)

preprocessed_texts = df['TEXT'].apply(preprocess_text)

# Create a dictionary from the preprocessed text
dictionary = corpora.Dictionary(preprocessed_texts)

# Create a bag-of-words corpus
corpus = [dictionary.doc2bow(text) for text in preprocessed_texts]

# Define the path to save the LDA model
lda_model_path = "lda_model_interactive.gensim"

# Check if the model already exists, if so, load it; otherwise, train and save it
if os.path.exists(lda_model_path):
    lda_model = models.LdaModel.load(lda_model_path)
    print("LDA model loaded from disk.")
else:
    lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)
    lda_model.save(lda_model_path)
    print("LDA model trained and saved to disk.")

# Label topics with the 2 most frequent bigrams
def identify_topics_by_frequent_bigrams(lda_model, corpus, preprocessed_texts, num_bigrams=2):
    """Identify each topic by the most frequent bigrams within the documents associated with that topic."""
    document_topics = [max(lda_model[doc], key=lambda x: x[1])[0] for doc in corpus]
    grouped_documents = {topic: [] for topic in range(lda_model.num_topics)}
    for doc_id, topic in enumerate(document_topics):
        grouped_documents[topic].append(doc_id)
    
    # Extract the most frequent bigrams for each topic
    topic_labels = {}
    for topic, documents in grouped_documents.items():
        group_texts = [preprocessed_texts[doc_id] for doc_id in documents]
        all_bigrams = [bigram for text in group_texts for bigram in ngrams(text, 2)]
        bigram_freq = Counter(all_bigrams)
        most_common_bigrams = [' '.join(bigram) for bigram, _ in bigram_freq.most_common(num_bigrams)]
        topic_labels[topic] = ' / '.join(most_common_bigrams)
    
    return topic_labels, grouped_documents

topic_labels, grouped_documents = identify_topics_by_frequent_bigrams(lda_model, corpus, preprocessed_texts)

# Prepare data for filtering and display
topic_data = []
for doc_id, row in df.iterrows():
    topics = lda_model.get_document_topics(corpus[doc_id])
    for topic_id, prob in topics:
        topic_data.append({
            "Document": doc_id,
            "Topic": topic_labels[topic_id],
            "Probability": prob,
            "Date": row['DATE'],
            "Text": row['TEXT']  # Assuming 'TEXT' is the column with the full news article text
        })

topic_df = pd.DataFrame(topic_data)

# Ensure the Date column is in datetime format
topic_df['Date'] = pd.to_datetime(topic_df['Date'])

# Dash app setup
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    html.H1("News Article Explorer by Topic"),
    
    dcc.Dropdown(
        id='topic-dropdown',
        options=[{'label': topic_labels[i], 'value': topic_labels[i]} for i in range(lda_model.num_topics)],
        value=topic_labels[0],
        clearable=False
    ),
    
    html.Div(id='news-display')
])

# Callback to update the displayed news articles based on the selected topic
@app.callback(
    Output('news-display', 'children'),
    [Input('topic-dropdown', 'value')]
)
def update_news_display(selected_topic):
    filtered_df = topic_df[(topic_df['Topic'] == selected_topic) & (topic_df['Probability'] > 0.5)]
    filtered_df = filtered_df.sort_values(by='Date', ascending=True)
    
    news_articles = []
    for _, row in filtered_df.iterrows():
        # Handle NaT values by using a placeholder or empty string
        date_str = row['Date'].strftime('%Y-%m-%d') if pd.notnull(row['Date']) else "Unknown Date"
        
        news_articles.append(html.Div([
            html.H3(f"Date: {date_str}"),
            html.P(row['Text']),
            html.Hr()
        ]))
    
    return news_articles

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andres\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andres\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Andres\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


LDA model loaded from disk.


: 